In [ ]:
#Assignment1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

df=pd.read_csv('uber.csv')
df.info()

# 1. Preprocess the data

df.shape

df.head()

df.isnull()

df.drop(columns=["Unnamed: 0","key"],inplace=True)
df.head()

df.isnull().sum()

df['dropoff_latitude'].fillna(value=df['dropoff_latitude'].mean, inplace=True)
df['dropoff_longitude'].fillna(value=df['dropoff_longitude'].median(), inplace=True)

df.isnull().sum()

df.dtypes

df.pickup_datetime=pd.to_datetime(df['pickup_datetime'])
df.dtypes

df=df.assign(hours = df.pickup_datetime.dt.hour,
            day = df.pickup_datetime.dt.day,
            month = df.pickup_datetime.dt.month,
            year = df.pickup_datetime.dt.year,
            dayofweek = df.pickup_datetime.dt.dayofweek)

df['dropoff_latitude'] = pd.to_numeric(df['dropoff_latitude'], errors='coerce')

df.dtypes

df

df=df.drop(["pickup_datetime"], axis=1)

df

# 2. Identify Outliers

df.plot(kind ="box" , subplots = True, layout=(6,2), figsize=(15,20))
plt.show()


# Define the functions to remove outliers
def remove_outlier(df1, col):
    Q1 = df1[col].quantile(0.25)
    Q3 = df1[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df1[col] = np.clip(df1[col], lower_bound, upper_bound)
    return df1

def treat_outliers_all(df1, col_list):
    for col in col_list:
        df1 = remove_outlier(df1, col)
    return df1

# List of columns you want to treat for outliers
columns_to_treat = df[['fare_amount', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count']]

# Call the function to treat outliers in all specified columns
df = treat_outliers_all(df, columns_to_treat)


df.plot(kind ="box" , subplots = True, layout=(6,2), figsize=(15,20))
plt.show()

# 3. Check the correlation

corr= df.corr()
corr

fig, axis =plt.subplots(figsize =(10,6))
sns.heatmap(df.corr(), annot=True)

# 4. Implement linear regression and random forestregression models.

df['dropoff_latitude'].fillna(df['dropoff_latitude'].mode()[0], inplace=True)


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


# Split the data into features and target
X = df[['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count']]
y = df['fare_amount']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_predictions = lr_model.predict(X_test)


# Random Forest Regression
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

# Evaluate the models
lr_rmse = mean_squared_error(y_test, lr_predictions, squared=False)
lr_r2 = r2_score(y_test, lr_predictions)

rf_rmse = mean_squared_error(y_test, rf_predictions, squared=False)
rf_r2 = r2_score(y_test, rf_predictions)

print("Linear Regression Results:")
print(f"RMSE: {lr_rmse}")
print(f"R-squared: {lr_r2}")
print("\nRandom Forest Regression Results:")
print(f"RMSE: {rf_rmse}")
print(f"R-squared: {rf_r2}")
